In [3]:
import pandas as pd
pd.reset_option('display.float_format')
pd.options.display.float_format = '{:.2f}'.format

df_ffca= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\FFCA.xlsx")
df_qv = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\QV.xlsx")
df_stock = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\stock.xlsx") 
df_stock.drop([0])
df_inventory = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\inventory.xlsx")
df_WC = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\Warehouse_Risk_Category.xlsx")



In [26]:
#column filter
cols_ffca = ['Purchase Assignment ID', 'Commodity', 'Inventory ID','Counterparty of Purchase', \
    'Purchase Settled Amount', 'Collateral Doc Type', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator', 'Collateral Doc Date', 'Location', 'Country']
cols_inventory = ['INVENTORY ID','COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'CITY']
cols_stock = ['Inventory Id','Origin Country']
cols_wc = ['LOCATION_DESCRIPTION', 'RISK_CATEGORY']

cols_ffca_blend = ['Purchase Assignment ID', 'Commodity', 'Inventory ID','Counterparty of Purchase', \
    'Purchase Settled Amount', 'Collateral Doc Type', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'QTY']

#NN
# cols_stock = ['Inventory Id','Origin Country', 'Inventory Quantity(Wet)', 'Delivery In Progress Qty', 'Receipt In Progress Qty (Wet)']

# df_inventory['INVENTORY ID'] = df_inventory['INVENTORY ID'].astype(str)
# df_stock['Inventory Id'] = df_stock['Inventory Id'].astype(str)
# df_ffca['Inventory ID'] = df_ffca['Inventory ID'].astype(str)

df_stock['Receipt In Progress Qty (Wet)'] = pd.to_numeric(df_stock['Receipt In Progress Qty (Wet)'], errors='coerce')
df_stock['Receipt In Progress Qty (Wet)'] = df_stock['Receipt In Progress Qty (Wet)'].fillna(0)


ndf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_WC_f1 = pd.DataFrame(df_WC, columns=cols_wc)
df_WC_f1['RISK_CATEGORY'] = df_WC_f1['RISK_CATEGORY'].map(lambda x: str(x)[13:])

#blend
bdf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca_blend)


In [27]:
def blend(a):
    return a == "Blend"

df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_nblend_inventory = df_inventory_f1.drop(row_list)

bffca_qv = pd.merge(bdf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
binventory_stock = pd.merge(df_blend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
bfinal_BB_WHS = pd.merge(bffca_qv, binventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
bfinal_BB_WHS = pd.merge(bfinal_BB_WHS,df_WC_f1, left_on = 'Location', right_on= 'LOCATION_DESCRIPTION', how='outer')


def function(a, b, c):
    c * (1 + (a-b)/a)

bfinal_BB_WHS['Purchased Wet Weight']  = bfinal_BB_WHS.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_BB_WHS = bfinal_BB_WHS.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])

#改名
bfinal_BB_WHS.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Authorized Paid Quantity', \
        'Financed Quantity', 'Inventory Current Value(USD)', 'Purchase Dry Weight' , 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Warehouse Reference', 'Warehouse Country', \
                    'INVENTORY ID',  'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'Warehouse City', 'Inventory Id','Purchase Material Origin Country',\
                         'LOCATION_DESCRIPTION', 'Warehouse Category',  'Purchase Wet Weight']

In [56]:
bfinal_BB_WHS['Collateral Reference'] = None
bfinal_BB_WHS['Purchase Dry Weight UOM'] = 'MT'
bfinal_BB_WHS['Purchase Wet Weight UOM'] = 'MT'
bfinal_BB_WHS['Receipt In Progress'] = 'N'
bfinal_BB_WHS['Delivery In Progress'] = 'N'
# bfinal_BB_WHS['Receipt In Progress'] = bfinal_BB_WHS.apply(lambda x: function(x['Receipt In Progress Qty (Wet)'], x['Inventory Quantity(Wet)']), axis = 1)
# bfinal_BB_WHS['Delivery In Progress'] = bfinal_BB_WHS.apply(lambda x: function(x['Delivery In Progress Qty'], x['Inventory Quantity(Wet)']), axis = 1)

bfinal_BB_WHS[['Authorized Paid Quantity', 'Financed Quantity']] = bfinal_BB_WHS[['Authorized Paid Quantity', 'Financed Quantity']].astype('float')
bfinal_BB_WHS.loc[:,"Financing Ratio"] = bfinal_BB_WHS["Authorized Paid Quantity"]/bfinal_BB_WHS["Financed Quantity"]

#复制列
bfinal_BB_WHS.loc[:, 'Eligible Dry Weight'] = bfinal_BB_WHS['Purchase Dry Weight']
bfinal_BB_WHS.loc[:, 'Eligible Wet Weight'] = bfinal_BB_WHS['Purchase Wet Weight']

#删除列
# bfinal_BB_WHS.drop(columns=['Inventory Quantity(Wet)', 'Delivery In Progress Qty', 'Receipt In Progress Qty (Wet)', 'LOCATION_DESCRIPTION'])
bfinal_BB_WHS.drop(columns=[ 'LOCATION_DESCRIPTION'])
#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty', 'Collateral Document Type', 'Collateral Reference', 'COLLATERAL DOCUMENT DATE', \
        'Purchase Commodity', 'Purchase Material Origin Country', 'Warehouse Reference', 'Warehouse Category', 'Warehouse Country', 'Warehouse City', \
             'Purchase Dry Weight', \
                'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)',\
                    'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorized Paid Quantity', \
                        'Financed Quantity','Financing Ratio', 'Inventory Current Value(USD)']

bfinal_BB_WHS_test = bfinal_BB_WHS[order]
bfinal_BB_WHS_test.head()

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Purchase Inventory ID,Purchase Traffic Operator,Purchase Contractual Counterparty,Collateral Document Type,Collateral Reference,COLLATERAL DOCUMENT DATE,...,Purchase Wet Weight UOM,Purchase Settlement Valuation(USD),Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight,Authorized Paid Quantity,Financed Quantity,Financing Ratio,Inventory Current Value(USD)
0,PTE,676816.00,676816.80,676816.8.1,878029.00,Gonzalo Machado,Angel Jorge Blacio Tinoco,Not Available,None,NaT,...,MT,-240000.00,N,N,0.10,None,0.00,0.10,0.00,-
1,PTE,676816.00,676816.80,676816.8.2,878039.00,Gonzalo Machado,Angel Jorge Blacio Tinoco,Not Available,None,NaT,...,MT,-140000.00,N,N,0.10,None,0.00,0.10,0.00,-
2,PTE,676816.00,676816.80,676816.8.3,878040.00,Gonzalo Machado,Angel Jorge Blacio Tinoco,Not Available,None,NaT,...,MT,-90000.00,N,N,0.10,None,0.00,0.10,0.00,-
3,PTE,676816.00,676816.80,676816.8.4,878044.00,Gonzalo Machado,Angel Jorge Blacio Tinoco,Not Available,None,NaT,...,MT,-30000.00,N,N,0.10,None,0.00,0.10,0.00,-
4,PTE,651548.00,651548.20,651548.20.1,881784.00,Gonzalo Machado,Empresa Minera Zambrano S.A. Eminzasa,Not Available,None,NaT,...,MT,0.00,N,N,0.10,None,0.00,0.00,nan,-


In [30]:
#改名
bfinal_BB_WHS.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Authorized Paid Quantity', \
        'Financed Quantity', 'Inventory Current Value(USD)', 'Purchase Dry Weight' , 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Warehouse Reference', 'Warehouse Country', \
                    'INVENTORY ID',  'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'Warehouse City', 'Inventory Id','Purchase Material Origin Country',\
                         'LOCATION_DESCRIPTION', 'Warehouse Category', 'Purchased Wet Weight']

In [31]:
nffca_qv = pd.merge(ndf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
ninventory_stock = pd.merge(df_nblend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
nfinal_BB_WHS = pd.merge(nffca_qv, ninventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
nfinal_BB_WHS = pd.merge(nfinal_BB_WHS,df_WC_f1, left_on = 'Location', right_on= 'LOCATION_DESCRIPTION', how='outer')

#改名
nfinal_BB_WHS.columns = ['Purchase Assignment Reference', 'Purchase Commodity', 'Purchase Inventory ID', 'Purchase Contractual Counterparty', \
    'Purchase Settlement Valuation(USD)', 'Collateral Document Type', 'Authorized Paid Quantity', \
        'Financed Quantity', 'Inventory Current Value(USD)', 'Assignment ID', 'Purchase Group Company Code', 'Purchase Trade ID', \
            'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Collateral Document Date', 'Warehouse Reference', 'Warehouse Country', \
                    'INVENTORY ID',  'COLLATERAL DOCUMENT DATE', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight', 'Warehouse City', 'Inventory Id','Purchase Material Origin Country',\
                         'LOCATION_DESCRIPTION', 'Warehouse Category']


# #新增固定列
# def function(a,b):
#     if a == b:
#         return 'N'
#     elif a == 0:
#         return 'N'
#     else:
#         return 'Y'
        
nfinal_BB_WHS['Collateral Reference'] = None
nfinal_BB_WHS['Purchase Dry Weight UOM'] = 'MT'
nfinal_BB_WHS['Purchase Wet Weight UOM'] = 'MT'
nfinal_BB_WHS['Receipt In Progress'] = 'N'
nfinal_BB_WHS['Delivery In Progress'] = 'N'
# nfinal_BB_WHS['Receipt In Progress'] = nfinal_BB_WHS.apply(lambda x: function(x['Receipt In Progress Qty (Wet)'], x['Inventory Quantity(Wet)']), axis = 1)
# nfinal_BB_WHS['Delivery In Progress'] = nfinal_BB_WHS.apply(lambda x: function(x['Delivery In Progress Qty'], x['Inventory Quantity(Wet)']), axis = 1)

nfinal_BB_WHS[['Authorized Paid Quantity', 'Financed Quantity']] = nfinal_BB_WHS[['Authorized Paid Quantity', 'Financed Quantity']].astype('float')
nfinal_BB_WHS.loc[:,"Financing Ratio"] = nfinal_BB_WHS["Authorized Paid Quantity"]/nfinal_BB_WHS["Financed Quantity"]

#复制列
nfinal_BB_WHS.loc[:, 'Eligible Dry Weight'] = nfinal_BB_WHS['Purchase Dry Weight']
nfinal_BB_WHS.loc[:, 'Eligible Wet Weight'] = nfinal_BB_WHS['Purchase Wet Weight']

#删除列
# nfinal_BB_WHS.drop(columns=['Inventory Quantity(Wet)', 'Delivery In Progress Qty', 'Receipt In Progress Qty (Wet)', 'LOCATION_DESCRIPTION'])
nfinal_BB_WHS.drop(columns=[ 'LOCATION_DESCRIPTION'])
#修改顺序
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Purchase Inventory ID', \
    'Purchase Traffic Operator', 'Purchase Contractual Counterparty', 'Collateral Document Type', 'Collateral Reference', 'COLLATERAL DOCUMENT DATE', \
        'Purchase Commodity', 'Purchase Material Origin Country', 'Warehouse Reference', 'Warehouse Category', 'Warehouse Country', 'Warehouse City', \
             'Purchase Dry Weight', \
                'Purchase Dry Weight UOM', 'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Purchase Settlement Valuation(USD)',\
                    'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight', 'Authorized Paid Quantity', \
                        'Financed Quantity','Financing Ratio', 'Inventory Current Value(USD)']

nfinal_BB_WHS_test = nfinal_BB_WHS[order]  

final_BB_WHS = pd.concat([nfinal_BB_WHS_test,bfinal_BB_WHS_test])

In [63]:
df_inventory_f1

,INVENTORY ID,COLLATERAL DOCUMENT DATE,PURCHASE ASSIGNMENT,DRY WEIGHT,WET WEIGHT,CITY
0,710356,NaT,669658.10.1,10000.00,11111.11,NaN
1,658771,2021-08-27,658612.7.5,519.72,520.29,Antwerpen
2,482406,2020-06-23,646526.15.1,926.82,988.78,Antwerp
3,618280,NaT,660289.15.1,0.00,0.00,NaN
4,592169,2021-03-18,Blend,9068.50,9998.00,Lianyungang
...,...,...,...,...,...,...
22449,773199,2022-06-19,661875.8.19,250.50,272.93,Fangcheng
22450,891713,NaT,686092.1.1,99.42,107.21,Manzanillo
22451,892249,NaT,685766.7.1,508.67,549.28,Manzanillo
22452,893259,NaT,685978.7.1,3913.90,4295.23,Manzanillo


In [33]:

nfinal_BB_WHS_test.head()

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Purchase Inventory ID,Purchase Traffic Operator,Purchase Contractual Counterparty,Collateral Document Type,Collateral Reference,COLLATERAL DOCUMENT DATE,...,Purchase Wet Weight UOM,Purchase Settlement Valuation(USD),Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight,Authorized Paid Quantity,Financed Quantity,Financing Ratio,Inventory Current Value(USD)
0,PTE,676816.00,676816.80,676816.8.1,878029.00,Gonzalo Machado,Angel Jorge Blacio Tinoco,Not Available,None,NaT,...,MT,-240000.00,N,N,0.10,0.10,0.00,0.10,0.00,-
1,PTE,676816.00,676816.80,676816.8.2,878039.00,Gonzalo Machado,Angel Jorge Blacio Tinoco,Not Available,None,NaT,...,MT,-140000.00,N,N,0.10,0.10,0.00,0.10,0.00,-
2,PTE,676816.00,676816.80,676816.8.3,878040.00,Gonzalo Machado,Angel Jorge Blacio Tinoco,Not Available,None,NaT,...,MT,-90000.00,N,N,0.10,0.10,0.00,0.10,0.00,-
3,PTE,676816.00,676816.80,676816.8.4,878044.00,Gonzalo Machado,Angel Jorge Blacio Tinoco,Not Available,None,NaT,...,MT,-30000.00,N,N,0.10,0.10,0.00,0.10,0.00,-
4,PTE,651548.00,651548.20,651548.20.1,881784.00,Gonzalo Machado,Empresa Minera Zambrano S.A. Eminzasa,Not Available,None,NaT,...,MT,0.00,N,N,0.10,0.10,0.00,0.00,nan,-


In [57]:

final_BB_WHS = pd.concat([nfinal_BB_WHS_test,bfinal_BB_WHS_test])
# result = pd.merge(final_BB_WHS, df_mia, left_on=["Purchase Assignment Reference"], right_on=["assignment"], how='right')
# result = result.drop_duplicates(['Purchase Assignment Reference'])
# result.head()
# result.to_excel(r"C:\Users\Yuyan.Lu\Desktop\\test6.xlsx")

In [27]:
import tkinter as tk
from os import path
from glob import glob  

#window creating
window = tk.Tk() 
window.title('BB reporter')
window.geometry('400x150')  # 这里的乘是小x

var_path = tk.StringVar()
entry_path = tk.Entry(window, textvariable=var_path, font=('Arial', 14))
entry_path.place(x=100,y=90)

def printing():
    path = var_path.get()
    
window.mainloop()